In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe as mp
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow

In [3]:
mp_face_mesh = mp.solutions.face_mesh
data = []
targets = []

directory = r"C:\Users\DELL\Desktop\ITI\DS & ALG\ml_project\AFLW2000"

for file_path in glob.glob(os.path.join(directory, "*.mat")):
    file_name = Path(file_path).stem
    
    image = cv2.imread(os.path.join(directory, f"{file_name}.jpg"))
    with mp_face_mesh.FaceMesh(static_image_mode=True) as faces:
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = faces.process(rgb_image)
        
        if results.multi_face_landmarks:
            image_landmarks = []
            for face_landmarks in results.multi_face_landmarks:
                
                for landmark in face_landmarks.landmark:
                    nose=face_landmarks.landmark[4]
                    chin=face_landmarks.landmark[152]
                    E_Dist=np.sqrt((nose.x-chin.x)**2+(nose.y-chin.y)**2+(nose.z-chin.z)**2)
                    relative_x = int(((landmark.x-nose.x)/(E_Dist)) * image.shape[1])
                    relative_y = int(((landmark.y-nose.y)/(E_Dist)) * image.shape[0])
                    # Append landmark coordinates to the list
                    image_landmarks.extend([relative_x, relative_y])
            
            # Load corresponding .mat file to extract roll, pitch, and yaw
            mat_data = sio.loadmat(os.path.join(directory, f"{file_name}.mat"))
            # Extract pose parameters (roll, pitch, yaw) and append to targets list
            pose_para = mat_data["Pose_Para"][0][:3]
            targets.append(pose_para)

            # Append facial landmarks to the data list
            data.append(image_landmarks)

# Convert lists to numpy arrays
data = np.array(data)
targets = np.array(targets)

# Convert data to a DataFrame
num_landmarks = data.shape[1] // 2  # Number of landmark points
columns = [f'X-{i//2 + 1}' if i % 2 == 0 else f'Y-{i//2 + 1}' for i in range(data.shape[1])]
df = pd.DataFrame(data, columns=columns)

# Add roll, pitch, and yaw columns to the DataFrame
df['roll'] = targets[:, 2]
df['pitch'] = targets[:, 0]
df['yaw'] = targets[:, 1]

# Save DataFrame to a CSV file
df.to_csv('face_LM.csv', index=False)


In [51]:
data=pd.read_csv('face_LM.csv')
data=data.dropna()
data

,X-1,Y-1,X-2,Y-2,X-3,Y-3,X-4,Y-4,X-5,Y-5,...,Y-465,X-466,Y-466,X-467,Y-467,X-468,Y-468,roll,pitch,yaw
0,-6,90,-1,25,-1,35,-14,-80,0,0,...,-176,69,-165,240,-205,254,-215,0.085676,-0.399231,0.018227
1,59,150,3,35,76,77,46,-73,0,0,...,-151,196,-143,390,-177,412,-188,0.300959,0.470065,1.189533
2,-12,108,0,29,-10,49,-35,-75,0,0,...,-161,24,-153,179,-188,193,-200,-0.236852,-0.184650,0.881137
3,72,97,13,28,47,36,-29,-81,0,0,...,-218,49,-204,230,-313,242,-332,-0.373374,-0.175379,0.299208
4,36,138,7,33,18,65,-34,-69,0,0,...,-157,28,-151,187,-217,202,-223,-0.220662,-0.026812,0.011965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,-34,131,-4,31,-28,55,-40,-81,0,0,...,-178,8,-170,160,-219,172,-228,0.038554,-0.306358,-0.283822
1849,-40,121,-5,31,-31,50,-37,-87,0,0,...,-190,10,-180,150,-224,163,-238,0.122791,-0.367547,-0.429723
1850,-43,131,-9,32,-29,66,-12,-79,0,0,...,-133,73,-129,234,-118,251,-132,-0.108536,-0.156035,0.567114
1851,-19,121,-4,32,-9,52,-21,-93,0,0,...,-194,75,-185,274,-218,292,-230,0.105118,-0.197102,-0.070430


In [7]:
# sub_cols=pd.DataFrame()
# sub_cols['X-4']=data['X-4']
# sub_cols['Y-4']=data['Y-4']


# for i in range(1, 469):
#     data[f'X-{i}'] -= sub_cols['X-4']
#     data[f'Y-{i}'] -= sub_cols['Y-4']
# data

In [6]:
# #scale the data
# chin_X=data['X-152']
# chin_Y=data['Y-152']
# E_Dist=np.sqrt((chin_X)**2+(chin_Y)**2)
# for i in range(1, 468):
#     data[f'X-{i}']=data[f'X-{i}']/E_Dist
#     data[f'Y-{i}']=data[f'Y-{i}']/E_Dist


# data

In [52]:
data_roll=data[data.drop(['pitch','yaw'],axis=1).columns]
data_pitch=data[data.drop(['roll','yaw'],axis=1).columns]
data_yaw=data[data.drop(['roll','pitch'],axis=1).columns]
data_roll.to_csv('face_LM_roll.csv', index=False)
data_pitch.to_csv('face_LM_pitch.csv', index=False)
data_yaw.to_csv('face_LM_yaw.csv', index=False)


In [53]:
data_roll=pd.read_csv('face_LM_roll.csv')
data_pitch=pd.read_csv('face_LM_pitch.csv')
data_yaw=pd.read_csv('face_LM_yaw.csv') 
data_roll

,X-1,Y-1,X-2,Y-2,X-3,Y-3,X-4,Y-4,X-5,Y-5,...,Y-464,X-465,Y-465,X-466,Y-466,X-467,Y-467,X-468,Y-468,roll
0,-6,90,-1,25,-1,35,-14,-80,0,0,...,-186,79,-176,69,-165,240,-205,254,-215,0.085676
1,59,150,3,35,76,77,46,-73,0,0,...,-162,221,-151,196,-143,390,-177,412,-188,0.300959
2,-12,108,0,29,-10,49,-35,-75,0,0,...,-172,30,-161,24,-153,179,-188,193,-200,-0.236852
3,72,97,13,28,47,36,-29,-81,0,0,...,-235,61,-218,49,-204,230,-313,242,-332,-0.373374
4,36,138,7,33,18,65,-34,-69,0,0,...,-168,37,-157,28,-151,187,-217,202,-223,-0.220662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,-34,131,-4,31,-28,55,-40,-81,0,0,...,-189,11,-178,8,-170,160,-219,172,-228,0.038554
1849,-40,121,-5,31,-31,50,-37,-87,0,0,...,-202,12,-190,10,-180,150,-224,163,-238,0.122791
1850,-43,131,-9,32,-29,66,-12,-79,0,0,...,-137,83,-133,73,-129,234,-118,251,-132,-0.108536
1851,-19,121,-4,32,-9,52,-21,-93,0,0,...,-204,87,-194,75,-185,274,-218,292,-230,0.105118


In [54]:
roll_features=data_roll[data_roll.drop(['roll'],axis=1).columns]
roll_target=data_roll['roll']

pitch_features=data_pitch[data_pitch.drop(['pitch'],axis=1).columns]
pitch_target=data_pitch['pitch']

yaw_features=data_yaw[data_yaw.drop(['yaw'],axis=1).columns]
yaw_target=data_yaw['yaw']

In [58]:
roll_features_train, roll_features_validation_test, roll_labels_train, roll_labels_validation_test = train_test_split(
    roll_features, roll_target, test_size=0.4, random_state=100)

roll_features_validation, roll_features_test, roll_labels_validation, roll_labels_test = train_test_split(
    roll_features_validation_test, roll_labels_validation_test, test_size=0.5, random_state=100)


pitch_features_train, pitch_features_validation_test, pitch_labels_train, pitch_labels_validation_test = train_test_split(
    pitch_features, pitch_target, test_size=0.4, random_state=100)

pitch_features_validation, pitch_features_test, pitch_labels_validation, pitch_labels_test = train_test_split(
    pitch_features_validation_test, pitch_labels_validation_test, test_size=0.5, random_state=100)


yaw_features_train, yaw_features_validation_test, yaw_labels_train, yaw_labels_validation_test = train_test_split(
    yaw_features, yaw_target, test_size=0.4, random_state=100)

yaw_features_validation, yaw_features_test, yaw_labels_validation, yaw_labels_test = train_test_split(
    yaw_features_validation_test, yaw_labels_validation_test, test_size=0.5, random_state=100)

In [80]:
svr_roll = SVR().fit(roll_features_train, roll_labels_train)

svr_pitch = SVR().fit(pitch_features_train, pitch_labels_train)


svr_yaw = SVR().fit(yaw_features_train, yaw_labels_train)

In [81]:
roll_train_score = svr_roll.score(roll_features_train, roll_labels_train)
roll_val_score = svr_roll.score(roll_features_validation, roll_labels_validation)
roll_test_scor = svr_roll.score(roll_features_test, roll_labels_test)

pitch_train_score = svr_pitch.score(pitch_features_train, pitch_labels_train)
pitch_val_score = svr_pitch.score(pitch_features_validation, pitch_labels_validation)
pitch_test_scor = svr_pitch.score(pitch_features_test, pitch_labels_test)

yaw_train_score = svr_yaw.score(yaw_features_train, yaw_labels_train)
yaw_val_score = svr_yaw.score(yaw_features_validation, yaw_labels_validation)
yaw_test_scor = svr_yaw.score(yaw_features_test, yaw_labels_test)

print("roll Train score: ", roll_train_score)
print("roll Validation score: ", roll_val_score)
print("roll Test score: ", roll_test_scor)

print("pitch Train score: ", pitch_train_score)
print("pitch Validation score: ", pitch_val_score)
print("pitch Test score: ", pitch_test_scor)

print("yaw Train score: ", yaw_train_score)
print("yaw Validation score: ", yaw_val_score)
print("yaw Test score: ", yaw_test_scor)

roll Train score:  0.8105649686856609
roll Validation score:  0.5992503568015927
roll Test score:  0.1623773816388263
pitch Train score:  0.6797095409166551
pitch Validation score:  0.6228789804321242
pitch Test score:  0.051300228270458326
yaw Train score:  0.9425518183595859
yaw Validation score:  0.909402964074205
yaw Test score:  0.6765428427545983


In [78]:
# param_grid = {
#     'kernel': ['rbf'],
#     'C': np.arange(0.1, 10, 0.2),
#     'epsilon': np.arange(0.01, 0.1, 0.002) 
# }
# # Initialize SVR model
# svr = SVR()

# # Perform GridSearchCV
# grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# # Fit the grid search to the data
# grid_search.fit(roll_features_train, roll_labels_train)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# print("Best Parameters:", best_params)
# print("Best Score (Negative Mean Squared Error):", best_score)

In [64]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [82]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

video = cv2.VideoCapture(0)

while True:
    ret, frame = video.read()
    if ret:
        results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                image_landmarks = []
                nose = face_landmarks.landmark[4]
                chin = face_landmarks.landmark[152]
                E_Dist = np.sqrt((nose.x - chin.x)**2 + (nose.y - chin.y)**2 + (nose.z - chin.z)**2)
                for landmark in face_landmarks.landmark:
                    relative_x = int(((landmark.x - nose.x) / E_Dist) * frame.shape[1])
                    relative_y = int(((landmark.y - nose.y) / E_Dist) * frame.shape[0])
                    image_landmarks.extend([relative_x, relative_y])

                image_landmarks = np.array(image_landmarks).reshape(1, -1)

                columns = [f'X-{i//2 + 1}' if i % 2 == 0 else f'Y-{i//2 + 1}' for i in range(image_landmarks.shape[1])]
                image_landmarks_df = pd.DataFrame(image_landmarks, columns=columns)

                # Predict roll, pitch, yaw
                pred_roll = svr_roll.predict(image_landmarks_df)
                pred_pitch = svr_pitch.predict(image_landmarks_df)
                pred_yaw = svr_yaw.predict(image_landmarks_df)

                draw_x = int(nose.x * frame.shape[1])
                draw_y = int(nose.y * frame.shape[0])

                frame = draw_axis(frame, pred_pitch, pred_yaw, pred_roll, draw_x, draw_y)

            cv2.imshow('Frame', frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    else:
        break

video.release()
cv2.destroyAllWindows()